In [23]:
pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

     |████████████████████████████████| 511.7 MB 7.8 kB/s eta 0:00:01     |█████████▎                      | 149.0 MB 3.3 MB/s eta 0:01:52     |██████████████████              | 287.4 MB 4.1 MB/s eta 0:00:55     |█████████████████████▏          | 338.6 MB 3.5 MB/s eta 0:00:50     |███████████████████████████████▋| 505.5 MB 3.5 MB/s eta 0:00:02
     |████████████████████████████████| 42 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 2.4 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.1 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 3.4 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=5f992188cabf5751253eff4572f179914031a060f5058720e07445f46ab484b4
  Stored in directory:

In [32]:
import pandas as pd
import keras_tuner
from tensorflow import keras 
from tensorflow.keras import layers
from keras_tuner import RandomSearch

In [25]:
df = pd.read_csv('Real_Combine.csv')

In [26]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [27]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

#### Hyperparameters
1. How many no of hidden layers we should have?
2. How many no. of neurons we should have in hidden layers?
3. What should be the learning rate?

In [37]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('unit_' + str(i),
                                           min_value = 32,
                                           max_value = 512,
                                           step = 32),
                              activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss = 'mean_absolute_error',
                    metrics=['mean_absolute_error'])
        return model
    

In [38]:
tuner = keras_tuner.RandomSearch(
           build_model,
           objective='val_mean_absolute_error',
           max_trials=5,
           executions_per_trial=3,
           directory='project',
           project_name='Air quality Index')

In [39]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [45]:
tuner.search(X_train, y_train,
             epochs=5, 
             validation_data = (X_test,y_test))

INFO:tensorflow:Oracle triggered exit


In [46]:
tuner.results_summary()

Results summary
Results in project/Air quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 7
unit_0: 64
learning_rate: 0.001
Score: nan
Trial summary
Hyperparameters:
num_layers: 19
unit_0: 384
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 14
unit_0: 352
learning_rate: 0.001
Score: nan
Trial summary
Hyperparameters:
num_layers: 17
unit_0: 416
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 7
unit_0: 448
learning_rate: 0.0001
Score: nan
